In [1]:
import pandas as pd
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Nhập dữ liệu

In [2]:
# Load data
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

# Gắn nhãn cho bộ dữ liệu

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return '0'
    elif rating == 3:
        return '1'
    elif rating in [4, 5]:
        return '2'
    else:
        return '3'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      Điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                      Bảo hành ít quá, chỉ 12 tháng         0
3            5                             Sản phẩm mượt, chạy êm         2
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


# Tiền xử lý dữ liệu

In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt         2
76              5                                    Rất ok         2
107             2                             pin tụt nhanh         0
135             5                              sản phẩm tốt         2
173             2                             hao pin nhanh         0
177             5                              sản phẩm tốt         2
181             5                                    Rất ok         2
183             5                              sản phẩm tốt         2
192             5                               sản phẩm ok         2
205             5                               máy dùng ok         2
225             4                                     

In [5]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                       bảo hành ít quá chỉ 12 tháng         0
3            5                              sản phẩm mượt chạy êm         2
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


In [6]:
# Lưu đánh giá vào cột X
X = df['Comment'].values.tolist()
# Lưu nhãn vào cột Y
y = df['Sentiment'].values.tolist()

# Tách từ, xây dựng bộ từ vựng và mã hóa

In [7]:
#Xây dựng hàm token từ underthesea
def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

In [8]:
vectorizer = TfidfVectorizer(tokenizer=tokenize_and_build_vocab_vietnamese, token_pattern=None,
                              max_features=5000, ngram_range=(1, 2), max_df=0.85, min_df=5)
# Áp dụng với cột Comment
X_tfidf = vectorizer.fit_transform(X)
vocabulary = vectorizer.get_feature_names_out()

# Mã hóa cho cột nhãn

In [9]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Cân bằng dữ liệu

In [10]:
#Sử dụng SMOTE để cân bằng dữ liệu

print('Trước khi cân bằng:', Counter(y))

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

print('Sau khi cân bằng:', Counter(y_resampled))


Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


In [11]:
print('Kích thước dữ liệu ban đầu:', X_tfidf.shape)
print('Kích thước dữ liệu sau khi cân bằng:', X_resampled.shape)
# In ra số lượng mẫu của từng nhãn trước và sau khi cân bằng
print('Trước khi cân bằng:', Counter(y))
print('Sau khi cân bằng:', Counter(y_resampled))

Kích thước dữ liệu ban đầu: (1498, 1200)
Kích thước dữ liệu sau khi cân bằng: (2496, 1200)
Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


# Chia tập dữ liệu

In [12]:
# Chia tập dữ liệu thành tập huấn luyện và tập validation (validation + test)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=42)

# Chia tập tạm thời thành tập validation và tập test cuối cùng
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.2, random_state=42)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

param_grid_nb = {
    'alpha': [0.1, 0.5, 1.0]
}

grid_search_nb = GridSearchCV(estimator=MultinomialNB(), param_grid=param_grid_nb, cv=5, n_jobs=-1, verbose=2)

grid_search_nb.fit(X_train, y_train)

best_nb = grid_search_nb.best_estimator_

print(f"Best parameters for Naive Bayes: {grid_search_nb.best_params_}")


Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters for Naive Bayes: {'alpha': 0.1}


In [14]:
best_nb = MultinomialNB(alpha=0.1)
best_nb.fit(X_train, y_train)

MultinomialNB(alpha=0.1)

In [15]:
# Dự đoán nhãn trên tập test
y_pred = best_nb.predict(X_test)

from sklearn.metrics import classification_report

# Đánh giá mô hình
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.88      0.87      0.87       209
           1       0.79      0.90      0.84       204
           2       0.97      0.83      0.90       211

    accuracy                           0.87       624
   macro avg       0.88      0.87      0.87       624
weighted avg       0.88      0.87      0.87       624

